In [1]:
import json
import csv
import tweepy
import re
import nltk
from nltk.stem import WordNetLemmatizer
import pandas as pd
import GetOldTweets3 as got
from nltk.corpus import stopwords
stopwords = set(stopwords.words('english'))
lemmatizer = WordNetLemmatizer()
import numpy as np

In [2]:
df=pd.read_csv('../../Datasets/Twitter/final_vit_withlikes.csv')
df

,Unnamed: 0,text,username,tweet_id,timestamp,hashtags,screen_name,retweets,likes,uni_handle,compound,Sentiment,score
0,0,Thanks to Developer Student Club-Vellore Insti...,Mohammad Shahid Ansari,1211639431959998464,2019-12-30 13:25:26,[],Shahid_Ansari_,0,0,0.0,0.4404,Positive,4
1,1,Junior Research Fellow In Vellore Institute Of...,Sarkari-Job,1211536334608904192,2019-12-30 06:35:46,['governmentjob'],sarkarijobind,0,0,0.0,0.0000,Neutral,3
2,2,VIT Notification 2020 – Opening for Various JR...,Govt Job Guide,1210822000945418240,2019-12-28 07:17:15,[],GovtJobGuide,0,0,0.0,0.3612,Positive,4
3,3,"Vallore Institute of Technology (VIT) Courses,...",Rajasthan Travel and Tourism Guide,1210652629606354944,2019-12-27 20:04:14,"['rajasthanpalaces', 'royalrajasthan', 'rajast...",e_rajasthan,0,0,0.0,0.0000,Neutral,3
4,4,MITS5509 Intelligent Systems for Analytics Ass...,Expertsmind.com,1209090557391208448,2019-12-23 12:37:07,"['MITS5509', 'IntelligentSystemsforAnalytics',...",ExpertsMindInc,0,0,0.0,0.7825,Positive,5
5,5,Thanks to Developer Student Club-Vellore Insti...,Mohammad Shahid Ansari,1211639431959998464,2019-12-30 13:25:26,[],Shahid_Ansari_,0,0,0.0,0.4404,Positive,4
6,6,Junior Research Fellow In Vellore Institute Of...,Sarkari-Job,1211536334608904192,2019-12-30 06:35:46,['governmentjob'],sarkarijobind,0,0,0.0,0.0000,Neutral,3
7,7,VIT Notification 2020 – Opening for Various JR...,Govt Job Guide,1210822000945418240,2019-12-28 07:17:15,[],GovtJobGuide,0,0,0.0,0.3612,Positive,4
8,8,"Vallore Institute of Technology (VIT) Courses,...",Rajasthan Travel and Tourism Guide,1210652629606354944,2019-12-27 20:04:14,"['rajasthanpalaces', 'royalrajasthan', 'rajast...",e_rajasthan,0,0,0.0,0.0000,Neutral,3
9,9,MITS5509 Intelligent Systems for Analytics Ass...,Expertsmind.com,1209090557391208448,2019-12-23 12:37:07,"['MITS5509', 'IntelligentSystemsforAnalytics',...",ExpertsMindInc,0,0,0.0,0.7825,Positive,5


In [3]:
df=df.drop(['Unnamed: 0'],axis=1)

In [4]:
def normalize(feature):
    max_value = df[feature].max()
    min_value = df[feature].min()
    result = (df[feature] - min_value) / (max_value - min_value)
    return result

In [5]:
df['retweets']=df['retweets'].astype(float)
df['likes']=df['likes'].astype(float)
df['retweets_n']=normalize('retweets')
df['likes_n']=normalize('likes')
df

,text,username,tweet_id,timestamp,hashtags,screen_name,retweets,likes,uni_handle,compound,Sentiment,score,retweets_n,likes_n
0,Thanks to Developer Student Club-Vellore Insti...,Mohammad Shahid Ansari,1211639431959998464,2019-12-30 13:25:26,[],Shahid_Ansari_,0.0,0.0,0.0,0.4404,Positive,4,0.000000,0.000000
1,Junior Research Fellow In Vellore Institute Of...,Sarkari-Job,1211536334608904192,2019-12-30 06:35:46,['governmentjob'],sarkarijobind,0.0,0.0,0.0,0.0000,Neutral,3,0.000000,0.000000
2,VIT Notification 2020 – Opening for Various JR...,Govt Job Guide,1210822000945418240,2019-12-28 07:17:15,[],GovtJobGuide,0.0,0.0,0.0,0.3612,Positive,4,0.000000,0.000000
3,"Vallore Institute of Technology (VIT) Courses,...",Rajasthan Travel and Tourism Guide,1210652629606354944,2019-12-27 20:04:14,"['rajasthanpalaces', 'royalrajasthan', 'rajast...",e_rajasthan,0.0,0.0,0.0,0.0000,Neutral,3,0.000000,0.000000
4,MITS5509 Intelligent Systems for Analytics Ass...,Expertsmind.com,1209090557391208448,2019-12-23 12:37:07,"['MITS5509', 'IntelligentSystemsforAnalytics',...",ExpertsMindInc,0.0,0.0,0.0,0.7825,Positive,5,0.000000,0.000000
5,Thanks to Developer Student Club-Vellore Insti...,Mohammad Shahid Ansari,1211639431959998464,2019-12-30 13:25:26,[],Shahid_Ansari_,0.0,0.0,0.0,0.4404,Positive,4,0.000000,0.000000
6,Junior Research Fellow In Vellore Institute Of...,Sarkari-Job,1211536334608904192,2019-12-30 06:35:46,['governmentjob'],sarkarijobind,0.0,0.0,0.0,0.0000,Neutral,3,0.000000,0.000000
7,VIT Notification 2020 – Opening for Various JR...,Govt Job Guide,1210822000945418240,2019-12-28 07:17:15,[],GovtJobGuide,0.0,0.0,0.0,0.3612,Positive,4,0.000000,0.000000
8,"Vallore Institute of Technology (VIT) Courses,...",Rajasthan Travel and Tourism Guide,1210652629606354944,2019-12-27 20:04:14,"['rajasthanpalaces', 'royalrajasthan', 'rajast...",e_rajasthan,0.0,0.0,0.0,0.0000,Neutral,3,0.000000,0.000000
9,MITS5509 Intelligent Systems for Analytics Ass...,Expertsmind.com,1209090557391208448,2019-12-23 12:37:07,"['MITS5509', 'IntelligentSystemsforAnalytics',...",ExpertsMindInc,0.0,0.0,0.0,0.7825,Positive,5,0.000000,0.000000


In [6]:
df['engagement_score']=60 * df['retweets_n'] + 40 * df['likes_n']

In [7]:
def clean_tweet(tweet): 
        ''' 
        Utility function to clean tweet text by removing links, special characters 
        using simple regex statements. 
        '''
        return ' '.join(re.sub("([^0-9A-Za-z \t]) |(\w+:\/\/\S+)", " ", tweet).split()) #(@[A-Za-z0-9]+)|

In [8]:
df['clean_tweets']=df['text'].map(clean_tweet)
df['text']=df['text'].map(str.lower)

In [9]:
from nltk.probability import FreqDist
from nltk import pos_tag, word_tokenize
from nltk.stem.porter import PorterStemmer
import matplotlib.pyplot as plt

from sklearn.manifold import MDS
from sklearn.metrics import euclidean_distances

In [10]:
df["clean_tweets"] = df["clean_tweets"].astype(str)
df["tweet_tokens"] = df["clean_tweets"].apply(lambda each_post: word_tokenize(re.sub(r'[^\w\s]',' ',each_post.lower())))
df["tweet_tokens"] = df["tweet_tokens"].apply(lambda list_of_words: [x for x in list_of_words if x not in stopwords])
df

,text,username,tweet_id,timestamp,hashtags,screen_name,retweets,likes,uni_handle,compound,Sentiment,score,retweets_n,likes_n,engagement_score,clean_tweets,tweet_tokens
0,thanks to developer student club-vellore insti...,Mohammad Shahid Ansari,1211639431959998464,2019-12-30 13:25:26,[],Shahid_Ansari_,0.0,0.0,0.0,0.4404,Positive,4,0.000000,0.000000,0.000000,Thanks to Developer Student Club-Vellore Insti...,"[thanks, developer, student, club, vellore, in..."
1,junior research fellow in vellore institute of...,Sarkari-Job,1211536334608904192,2019-12-30 06:35:46,['governmentjob'],sarkarijobind,0.0,0.0,0.0,0.0000,Neutral,3,0.000000,0.000000,0.000000,Junior Research Fellow In Vellore Institute Of...,"[junior, research, fellow, vellore, institute,..."
2,vit notification 2020 – opening for various jr...,Govt Job Guide,1210822000945418240,2019-12-28 07:17:15,[],GovtJobGuide,0.0,0.0,0.0,0.3612,Positive,4,0.000000,0.000000,0.000000,VIT Notification 2020 Opening for Various JRF ...,"[vit, notification, 2020, opening, various, jr..."
3,"vallore institute of technology (vit) courses,...",Rajasthan Travel and Tourism Guide,1210652629606354944,2019-12-27 20:04:14,"['rajasthanpalaces', 'royalrajasthan', 'rajast...",e_rajasthan,0.0,0.0,0.0,0.0000,Neutral,3,0.000000,0.000000,0.000000,Vallore Institute of Technology (VIT Courses F...,"[vallore, institute, technology, vit, courses,..."
4,mits5509 intelligent systems for analytics ass...,Expertsmind.com,1209090557391208448,2019-12-23 12:37:07,"['MITS5509', 'IntelligentSystemsforAnalytics',...",ExpertsMindInc,0.0,0.0,0.0,0.7825,Positive,5,0.000000,0.000000,0.000000,MITS5509 Intelligent Systems for Analytics Ass...,"[mits5509, intelligent, systems, analytics, as..."
5,thanks to developer student club-vellore insti...,Mohammad Shahid Ansari,1211639431959998464,2019-12-30 13:25:26,[],Shahid_Ansari_,0.0,0.0,0.0,0.4404,Positive,4,0.000000,0.000000,0.000000,Thanks to Developer Student Club-Vellore Insti...,"[thanks, developer, student, club, vellore, in..."
6,junior research fellow in vellore institute of...,Sarkari-Job,1211536334608904192,2019-12-30 06:35:46,['governmentjob'],sarkarijobind,0.0,0.0,0.0,0.0000,Neutral,3,0.000000,0.000000,0.000000,Junior Research Fellow In Vellore Institute Of...,"[junior, research, fellow, vellore, institute,..."
7,vit notification 2020 – opening for various jr...,Govt Job Guide,1210822000945418240,2019-12-28 07:17:15,[],GovtJobGuide,0.0,0.0,0.0,0.3612,Positive,4,0.000000,0.000000,0.000000,VIT Notification 2020 Opening for Various JRF ...,"[vit, notification, 2020, opening, various, jr..."
8,"vallore institute of technology (vit) courses,...",Rajasthan Travel and Tourism Guide,1210652629606354944,2019-12-27 20:04:14,"['rajasthanpalaces', 'royalrajasthan', 'rajast...",e_rajasthan,0.0,0.0,0.0,0.0000,Neutral,3,0.000000,0.000000,0.000000,Vallore Institute of Technology (VIT Courses F...,"[vallore, institute, technology, vit, courses,..."
9,mits5509 intelligent systems for analytics ass...,Expertsmind.com,1209090557391208448,2019-12-23 12:37:07,"['MITS5509', 'IntelligentSystemsforAnalytics',...",ExpertsMindInc,0.0,0.0,0.0,0.7825,Positive,5,0.000000,0.000000,0.000000,MITS5509 Intelligent Systems for Analytics Ass...,"[mits5509, intelligent, systems, analytics, as..."


In [11]:
df['pos']=df['tweet_tokens'].apply(nltk.tag.pos_tag)
df

,text,username,tweet_id,timestamp,hashtags,screen_name,retweets,likes,uni_handle,compound,Sentiment,score,retweets_n,likes_n,engagement_score,clean_tweets,tweet_tokens,pos
0,thanks to developer student club-vellore insti...,Mohammad Shahid Ansari,1211639431959998464,2019-12-30 13:25:26,[],Shahid_Ansari_,0.0,0.0,0.0,0.4404,Positive,4,0.000000,0.000000,0.000000,Thanks to Developer Student Club-Vellore Insti...,"[thanks, developer, student, club, vellore, in...","[(thanks, NNS), (developer, VBP), (student, NN..."
1,junior research fellow in vellore institute of...,Sarkari-Job,1211536334608904192,2019-12-30 06:35:46,['governmentjob'],sarkarijobind,0.0,0.0,0.0,0.0000,Neutral,3,0.000000,0.000000,0.000000,Junior Research Fellow In Vellore Institute Of...,"[junior, research, fellow, vellore, institute,...","[(junior, JJ), (research, NN), (fellow, NN), (..."
2,vit notification 2020 – opening for various jr...,Govt Job Guide,1210822000945418240,2019-12-28 07:17:15,[],GovtJobGuide,0.0,0.0,0.0,0.3612,Positive,4,0.000000,0.000000,0.000000,VIT Notification 2020 Opening for Various JRF ...,"[vit, notification, 2020, opening, various, jr...","[(vit, NN), (notification, NN), (2020, CD), (o..."
3,"vallore institute of technology (vit) courses,...",Rajasthan Travel and Tourism Guide,1210652629606354944,2019-12-27 20:04:14,"['rajasthanpalaces', 'royalrajasthan', 'rajast...",e_rajasthan,0.0,0.0,0.0,0.0000,Neutral,3,0.000000,0.000000,0.000000,Vallore Institute of Technology (VIT Courses F...,"[vallore, institute, technology, vit, courses,...","[(vallore, IN), (institute, JJ), (technology, ..."
4,mits5509 intelligent systems for analytics ass...,Expertsmind.com,1209090557391208448,2019-12-23 12:37:07,"['MITS5509', 'IntelligentSystemsforAnalytics',...",ExpertsMindInc,0.0,0.0,0.0,0.7825,Positive,5,0.000000,0.000000,0.000000,MITS5509 Intelligent Systems for Analytics Ass...,"[mits5509, intelligent, systems, analytics, as...","[(mits5509, JJ), (intelligent, JJ), (systems, ..."
5,thanks to developer student club-vellore insti...,Mohammad Shahid Ansari,1211639431959998464,2019-12-30 13:25:26,[],Shahid_Ansari_,0.0,0.0,0.0,0.4404,Positive,4,0.000000,0.000000,0.000000,Thanks to Developer Student Club-Vellore Insti...,"[thanks, developer, student, club, vellore, in...","[(thanks, NNS), (developer, VBP), (student, NN..."
6,junior research fellow in vellore institute of...,Sarkari-Job,1211536334608904192,2019-12-30 06:35:46,['governmentjob'],sarkarijobind,0.0,0.0,0.0,0.0000,Neutral,3,0.000000,0.000000,0.000000,Junior Research Fellow In Vellore Institute Of...,"[junior, research, fellow, vellore, institute,...","[(junior, JJ), (research, NN), (fellow, NN), (..."
7,vit notification 2020 – opening for various jr...,Govt Job Guide,1210822000945418240,2019-12-28 07:17:15,[],GovtJobGuide,0.0,0.0,0.0,0.3612,Positive,4,0.000000,0.000000,0.000000,VIT Notification 2020 Opening for Various JRF ...,"[vit, notification, 2020, opening, various, jr...","[(vit, NN), (notification, NN), (2020, CD), (o..."
8,"vallore institute of technology (vit) courses,...",Rajasthan Travel and Tourism Guide,1210652629606354944,2019-12-27 20:04:14,"['rajasthanpalaces', 'royalrajasthan', 'rajast...",e_rajasthan,0.0,0.0,0.0,0.0000,Neutral,3,0.000000,0.000000,0.000000,Vallore Institute of Technology (VIT Courses F...,"[vallore, institute, technology, vit, courses,...","[(vallore, IN), (institute, JJ), (technology, ..."
9,mits5509 intelligent systems for analytics ass...,Expertsmind.com,1209090557391208448,2019-12-23 12:37:07,"['MITS5509', 'IntelligentSystemsforAnalytics',...",ExpertsMindInc,0.0,0.0,0.0,0.7825,Positive,5,0.000000,0.000000,0.000000,MITS5509 Intelligent Systems for Analytics Ass...,"[mits5509, intelligent, systems, analytics, as...","[(mits5509, JJ), (intelligent, JJ), (systems, ..."


In [12]:
# Extract the list of nouns
df['nouns']=df['pos'].apply(lambda pos_list: [x[0] for x in pos_list if x[1].startswith('NN')])
df

,text,username,tweet_id,timestamp,hashtags,screen_name,retweets,likes,uni_handle,compound,Sentiment,score,retweets_n,likes_n,engagement_score,clean_tweets,tweet_tokens,pos,nouns
0,thanks to developer student club-vellore insti...,Mohammad Shahid Ansari,1211639431959998464,2019-12-30 13:25:26,[],Shahid_Ansari_,0.0,0.0,0.0,0.4404,Positive,4,0.000000,0.000000,0.000000,Thanks to Developer Student Club-Vellore Insti...,"[thanks, developer, student, club, vellore, in...","[(thanks, NNS), (developer, VBP), (student, NN...","[thanks, student, club, technology, dsc, vit, ..."
1,junior research fellow in vellore institute of...,Sarkari-Job,1211536334608904192,2019-12-30 06:35:46,['governmentjob'],sarkarijobind,0.0,0.0,0.0,0.0000,Neutral,3,0.000000,0.000000,0.000000,Junior Research Fellow In Vellore Institute Of...,"[junior, research, fellow, vellore, institute,...","[(junior, JJ), (research, NN), (fellow, NN), (...","[research, fellow, institute, technology, univ..."
2,vit notification 2020 – opening for various jr...,Govt Job Guide,1210822000945418240,2019-12-28 07:17:15,[],GovtJobGuide,0.0,0.0,0.0,0.3612,Positive,4,0.000000,0.000000,0.000000,VIT Notification 2020 Opening for Various JRF ...,"[vit, notification, 2020, opening, various, jr...","[(vit, NN), (notification, NN), (2020, CD), (o...","[vit, notification, jrf, posts, technology, no..."
3,"vallore institute of technology (vit) courses,...",Rajasthan Travel and Tourism Guide,1210652629606354944,2019-12-27 20:04:14,"['rajasthanpalaces', 'royalrajasthan', 'rajast...",e_rajasthan,0.0,0.0,0.0,0.0000,Neutral,3,0.000000,0.000000,0.000000,Vallore Institute of Technology (VIT Courses F...,"[vallore, institute, technology, vit, courses,...","[(vallore, IN), (institute, JJ), (technology, ...","[technology, vit, courses, fees, process, raja..."
4,mits5509 intelligent systems for analytics ass...,Expertsmind.com,1209090557391208448,2019-12-23 12:37:07,"['MITS5509', 'IntelligentSystemsforAnalytics',...",ExpertsMindInc,0.0,0.0,0.0,0.7825,Positive,5,0.000000,0.000000,0.000000,MITS5509 Intelligent Systems for Analytics Ass...,"[mits5509, intelligent, systems, analytics, as...","[(mits5509, JJ), (intelligent, JJ), (systems, ...","[systems, analytics, help, institute, technolo..."
5,thanks to developer student club-vellore insti...,Mohammad Shahid Ansari,1211639431959998464,2019-12-30 13:25:26,[],Shahid_Ansari_,0.0,0.0,0.0,0.4404,Positive,4,0.000000,0.000000,0.000000,Thanks to Developer Student Club-Vellore Insti...,"[thanks, developer, student, club, vellore, in...","[(thanks, NNS), (developer, VBP), (student, NN...","[thanks, student, club, technology, dsc, vit, ..."
6,junior research fellow in vellore institute of...,Sarkari-Job,1211536334608904192,2019-12-30 06:35:46,['governmentjob'],sarkarijobind,0.0,0.0,0.0,0.0000,Neutral,3,0.000000,0.000000,0.000000,Junior Research Fellow In Vellore Institute Of...,"[junior, research, fellow, vellore, institute,...","[(junior, JJ), (research, NN), (fellow, NN), (...","[research, fellow, institute, technology, univ..."
7,vit notification 2020 – opening for various jr...,Govt Job Guide,1210822000945418240,2019-12-28 07:17:15,[],GovtJobGuide,0.0,0.0,0.0,0.3612,Positive,4,0.000000,0.000000,0.000000,VIT Notification 2020 Opening for Various JRF ...,"[vit, notification, 2020, opening, various, jr...","[(vit, NN), (notification, NN), (2020, CD), (o...","[vit, notification, jrf, posts, technology, no..."
8,"vallore institute of technology (vit) courses,...",Rajasthan Travel and Tourism Guide,1210652629606354944,2019-12-27 20:04:14,"['rajasthanpalaces', 'royalrajasthan', 'rajast...",e_rajasthan,0.0,0.0,0.0,0.0000,Neutral,3,0.000000,0.000000,0.000000,Vallore Institute of Technology (VIT Courses F...,"[vallore, institute, technology, vit, courses,...","[(vallore, IN), (institute, JJ), (technology, ...","[technology, vit, courses, fees, process, raja..."
9,mits5509 intelligent systems for analytics ass...,Expertsmind.com,1209090557391208448,2019-12-23 12:37:07,"['MIT

In [13]:
#Filter the noun list only to make the list of attributes 
noun_lst = df['nouns'].apply(pd.Series).stack().unique()

# # get the frequencies for the nouns:
noun_df= pd.DataFrame(noun_lst)
noun_df.columns=['nouns']
noun_df

,nouns
0,thanks
1,student
2,club
3,technology
4,dsc
5,vit
6,google
7,developers
8,group
9,machine


In [14]:
## get frequency of all words
all_words = df['nouns'].sum()
freq_dist = nltk.FreqDist(all_words)

df_fdist=pd.DataFrame(list(freq_dist.items()), columns=['term', 'freq'])

noun_freq_df = pd.merge(noun_df, df_fdist, right_on='term', left_on='nouns')
noun_freq_df

,nouns,term,freq
0,thanks,thanks,16
1,student,student,50
2,club,club,6
3,technology,technology,1264
4,dsc,dsc,2
5,vit,vit,464
6,google,google,4
7,developers,developers,2
8,group,group,6
9,machine,machine,16


In [15]:
noun_freq_df=noun_freq_df.sort_values(by=['freq'], ascending=False)
noun_freq_df

,nouns,term,freq
3,technology,technology,1264
16,institute,institute,708
17,university,university,551
35,twitter,twitter,544
45,com,com,538
5,vit,vit,464
216,institutions,institutions,218
857,side,side,213
246,investment,investment,213
642,premises,premises,212


In [16]:
noun_freq_df.to_csv('nouns_tweets.csv')

In [17]:
#list of irreleveant tweet topics
l=['victorianinstituteoftechnology','bennettunivpic','rajasthaniculture','cultureofrajasthan','rajasthantourism','rajasthanpalaces','australiapic']

In [18]:
df['irrelevance']=df['nouns'].apply(lambda x: len(set(x).intersection(l)))
df[df.irrelevance!=0]

,text,username,tweet_id,timestamp,hashtags,screen_name,retweets,likes,uni_handle,compound,Sentiment,score,retweets_n,likes_n,engagement_score,clean_tweets,tweet_tokens,pos,nouns,irrelevance
3,"vallore institute of technology (vit) courses,...",Rajasthan Travel and Tourism Guide,1210652629606354944,2019-12-27 20:04:14,"['rajasthanpalaces', 'royalrajasthan', 'rajast...",e_rajasthan,0.0,0.0,0.0,0.0000,Neutral,3,0.000000,0.000000,0.000000,Vallore Institute of Technology (VIT Courses F...,"[vallore, institute, technology, vit, courses,...","[(vallore, IN), (institute, JJ), (technology, ...","[technology, vit, courses, fees, process, raja...",4
4,mits5509 intelligent systems for analytics ass...,Expertsmind.com,1209090557391208448,2019-12-23 12:37:07,"['MITS5509', 'IntelligentSystemsforAnalytics',...",ExpertsMindInc,0.0,0.0,0.0,0.7825,Positive,5,0.000000,0.000000,0.000000,MITS5509 Intelligent Systems for Analytics Ass...,"[mits5509, intelligent, systems, analytics, as...","[(mits5509, JJ), (intelligent, JJ), (systems, ...","[systems, analytics, help, institute, technolo...",2
8,"vallore institute of technology (vit) courses,...",Rajasthan Travel and Tourism Guide,1210652629606354944,2019-12-27 20:04:14,"['rajasthanpalaces', 'royalrajasthan', 'rajast...",e_rajasthan,0.0,0.0,0.0,0.0000,Neutral,3,0.000000,0.000000,0.000000,Vallore Institute of Technology (VIT Courses F...,"[vallore, institute, technology, vit, courses,...","[(vallore, IN), (institute, JJ), (technology, ...","[technology, vit, courses, fees, process, raja...",4
9,mits5509 intelligent systems for analytics ass...,Expertsmind.com,1209090557391208448,2019-12-23 12:37:07,"['MITS5509', 'IntelligentSystemsforAnalytics',...",ExpertsMindInc,0.0,0.0,0.0,0.7825,Positive,5,0.000000,0.000000,0.000000,MITS5509 Intelligent Systems for Analytics Ass...,"[mits5509, intelligent, systems, analytics, as...","[(mits5509, JJ), (intelligent, JJ), (systems, ...","[systems, analytics, help, institute, technolo...",2
13,"need itne2003 install, configure, operate and ...",Expertsmind.com,1135496099634900992,2019-06-03 10:39:01,"['ITNE2003', 'Configure', 'MediumSize', 'Netwo...",ExpertsMindInc,0.0,0.0,0.0,0.5423,Positive,4,0.000000,0.000000,0.000000,Need ITNE2003 Install Configure Operate and tr...,"[need, itne2003, install, configure, operate, ...","[(need, NN), (itne2003, VBZ), (install, JJ), (...","[need, configure, troubleshoot, medium, size, ...",1
14,do you need itap1004 website development assig...,Expertsmind.com,1135490065885306880,2019-06-03 10:15:02,"['ITAP1004', 'WebsiteDevelopment', 'VictorianI...",ExpertsMindInc,0.0,0.0,0.0,0.8264,Positive,5,0.000000,0.000000,0.000000,Do you need ITAP1004 Website Development assig...,"[need, itap1004, website, development, assignm...","[(need, NN), (itap1004, VBZ), (website, JJ), (...","[need, development, assignment, help, institut...",1
15,looking for itne2005 develop a security infras...,Expertsmind.com,1135440044473311232,2019-06-03 06:56:16,"['ITNE2005', 'DevelopasecurityInfrastructure',...",ExpertsMindInc,0.0,0.0,0.0,0.6249,Positive,5,0.000000,0.000000,0.000000,Looking for ITNE2005 Develop a security Infras...,"[looking, itne2005, develop, security, infrast...","[(looking, VBG), (itne2005, NN), (develop, VB)...","[itne2005, security, infrastructure, medium, s...",1
21,"need itne2003 install, configure, operate and ...",Expertsmind.com,1135496099634900992,2019-06-03 10:39:01,"['ITNE2003', 'Configure', 'MediumSize', 'Netwo...",ExpertsMindInc,0.0,0.0,0.0,0.5423,Positive,4,0.000000,0.000000,0.000000,Need ITNE2003 Install Configure Operate and tr...,"[need, itne2003, install, configure, operate, ...","[(need, NN), (itne2003, VBZ), (install, JJ), (...","[need, configure, troubleshoot, medium, size, ...",1
22,do you need itap1004 website development assig...,Expertsmind.com,1135490065885306880,2019-06-03 10:15:02,"['ITAP1004', 'WebsiteDevelopment', 'VictorianI...",ExpertsMindInc,0.0,0.0,0.0,0.8264,Positive,5,0.000000,0.000000,0.000000,Do you need ITA

In [19]:
def assignpred(n):
    if n==2:
        return 1
    if n==4:
        return 5
    return n

In [20]:
engagement_median = df['engagement_score'].median()
df["engagement"] = df['engagement_score'].apply(lambda x: 1 if x > engagement_median else 0)

In [21]:
df['new_score']=df['score']
df_eng1=df[df.engagement==1]
df_eng1['new_score']=df_eng1['score'].apply(lambda x: assignpred(x))

/Users/Manam/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [22]:
df[df.engagement==1]=df_eng1

In [23]:
#if the tweet is by university, assign sentiment as neutral
df.loc[df['uni_handle']==1,'new_score']=3

In [24]:
#if the tweet contains irrelevant terms, assign sentiment as neutral
df.loc[df['irrelevance']!=0,'new_score']=3

In [25]:
df.rename(columns = {'new_score':'sentiment_score'}, inplace = True) 
df

,text,username,tweet_id,timestamp,hashtags,screen_name,retweets,likes,uni_handle,compound,...,retweets_n,likes_n,engagement_score,clean_tweets,tweet_tokens,pos,nouns,irrelevance,engagement,sentiment_score
0,thanks to developer student club-vellore insti...,Mohammad Shahid Ansari,1211639431959998464,2019-12-30 13:25:26,[],Shahid_Ansari_,0.0,0.0,0.0,0.4404,...,0.000000,0.000000,0.000000,Thanks to Developer Student Club-Vellore Insti...,"[thanks, developer, student, club, vellore, in...","[(thanks, NNS), (developer, VBP), (student, NN...","[thanks, student, club, technology, dsc, vit, ...",0,0,4
1,junior research fellow in vellore institute of...,Sarkari-Job,1211536334608904192,2019-12-30 06:35:46,['governmentjob'],sarkarijobind,0.0,0.0,0.0,0.0000,...,0.000000,0.000000,0.000000,Junior Research Fellow In Vellore Institute Of...,"[junior, research, fellow, vellore, institute,...","[(junior, JJ), (research, NN), (fellow, NN), (...","[research, fellow, institute, technology, univ...",0,0,3
2,vit notification 2020 – opening for various jr...,Govt Job Guide,1210822000945418240,2019-12-28 07:17:15,[],GovtJobGuide,0.0,0.0,0.0,0.3612,...,0.000000,0.000000,0.000000,VIT Notification 2020 Opening for Various JRF ...,"[vit, notification, 2020, opening, various, jr...","[(vit, NN), (notification, NN), (2020, CD), (o...","[vit, notification, jrf, posts, technology, no...",0,0,4
3,"vallore institute of technology (vit) courses,...",Rajasthan Travel and Tourism Guide,1210652629606354944,2019-12-27 20:04:14,"['rajasthanpalaces', 'royalrajasthan', 'rajast...",e_rajasthan,0.0,0.0,0.0,0.0000,...,0.000000,0.000000,0.000000,Vallore Institute of Technology (VIT Courses F...,"[vallore, institute, technology, vit, courses,...","[(vallore, IN), (institute, JJ), (technology, ...","[technology, vit, courses, fees, process, raja...",4,0,3
4,mits5509 intelligent systems for analytics ass...,Expertsmind.com,1209090557391208448,2019-12-23 12:37:07,"['MITS5509', 'IntelligentSystemsforAnalytics',...",ExpertsMindInc,0.0,0.0,0.0,0.7825,...,0.000000,0.000000,0.000000,MITS5509 Intelligent Systems for Analytics Ass...,"[mits5509, intelligent, systems, analytics, as...","[(mits5509, JJ), (intelligent, JJ), (systems, ...","[systems, analytics, help, institute, technolo...",2,0,3
5,thanks to developer student club-vellore insti...,Mohammad Shahid Ansari,1211639431959998464,2019-12-30 13:25:26,[],Shahid_Ansari_,0.0,0.0,0.0,0.4404,...,0.000000,0.000000,0.000000,Thanks to Developer Student Club-Vellore Insti...,"[thanks, developer, student, club, vellore, in...","[(thanks, NNS), (developer, VBP), (student, NN...","[thanks, student, club, technology, dsc, vit, ...",0,0,4
6,junior research fellow in vellore institute of...,Sarkari-Job,1211536334608904192,2019-12-30 06:35:46,['governmentjob'],sarkarijobind,0.0,0.0,0.0,0.0000,...,0.000000,0.000000,0.000000,Junior Research Fellow In Vellore Institute Of...,"[junior, research, fellow, vellore, institute,...","[(junior, JJ), (research, NN), (fellow, NN), (...","[research, fellow, institute, technology, univ...",0,0,3
7,vit notification 2020 – opening for various jr...,Govt Job Guide,1210822000945418240,2019-12-28 07:17:15,[],GovtJobGuide,0.0,0.0,0.0,0.3612,...,0.000000,0.000000,0.000000,VIT Notification 2020 Opening for Various JRF ...,"[vit, notification, 2020, opening, various, jr...","[(vit, NN), (notification, NN), (2020, CD), (o...","[vit, notification, jrf, posts, technology, no...",0,0,4
8,"vallore institute of technology (vit) courses,...",Rajasthan Travel and Tourism Guide,1210652629606354944,2019-12-27 20:04:14,"['rajasthanpalaces', 'royalrajasthan', 'rajast...",e_rajasthan,0.0,0.0,0.0,0.0000,...,0.000000,0.000000,0.000000,Vallore Institute of Technology (VIT Courses F...,"[vallore, institute, technology, vit, courses,...","[(vallore, IN), (institute, JJ), (technology, ...","[technology, vit, courses, fees, process, raja...",4,0,3
9,mits5509 intelligent systems for analytics ass...,Expertsmind.com,1209090557391208448,2019

In [26]:
df.to_csv('../../Datasets/Twitter/VITWeightedSentiment.csv')